# Equality & Inheritance

## Simmetria 
L'implementazione *ovvia* non funziona: se un sottotipo aggiunge un "valore", non basta controllare con `instanceof` e verificare che siano uguali i valori in gioco…

In [1]:
class T0 {
    private final int a;
    public T0(int a) { this.a = a; }
    public boolean equals(Object o) {
        if (o instanceof T0) {
            final T0 t = (T0)o;
            return a == t.a;
        }
        return false;
    }
}

class S0 extends T0 {
    private final int b;
    public S0(int a, int b) { super(a); this.b = b; }
    public boolean equals(Object o) {
        if (o instanceof S0) {
            final S0 s = (S0)o;
            return super.equals(o) && b == s.b;
        }
        return false;
    }
}

S0 s = new S0(1, 2);
T0 t = new T0(1);

Viene violata la proprietà di **simmetria**, ossia $t \sim s$ non implica $s \sim t$

In [2]:
t.equals(s)

true

In [3]:
s.equals(t)

false

Quel che accade è che `t instanceof S0` è *falso*, ma `s instanceof T0` è *vero*, per cui l'implementazione di `equals` di `S0` non sa cosa fare con `t`, mentre quella di `T0` funziona con `s` (ovviamente trascurando `b` di cui ignora l'esistenza).

## Transitività

Non è facile aggiustare l'implementazione di `equals` di `S1` gestendo accortamente il caso in cui `o` ha il tipo effettivo di `T1` imitando il suo comportamento, ossai "scordandosi" di `b`.

In [4]:
class T1 {
    private final int a;
    public T1(int a) { this.a = a; }
    public boolean equals(Object o) {
        if (o instanceof T1) {
            final T1 t = (T1)o;
            return a == t.a;
        }
        return false;
    }
}

class S1 extends T1 {
    private final int b;
    public S1(int a, int b) { super(a); this.b = b; }
    public boolean equals(Object o) {
        if (o instanceof S1) {
            final S1 s = (S1)o;
            return super.equals(o) && b == s.b;
        }
        if (o instanceof T1) return super.equals(o);
        return false;
    }
}

S1 s = new S1(1, 2);
T1 t = new T1(1);
S1 u = new S1(1, 3);

Così facendo infatti, la simmetria è rispsttata

In [5]:
t.equals(s);

true

In [6]:
s.equals(t);

true

Ma ovviamente risulterà violata la proprietà **transitiva**, ovvero anche avendo $s \sim t$ e $t \sim u$ non è detto che $s \sim u$

In [7]:
t.equals(u);

true

In [8]:
s.equals(u);

false

## Il primcipio di sostituione

Si potrebbe pensare che il problema sia `instanceof` che è antisimmetrico, per cui una soluzione che viene talvolta suggerita è di sostituire nell'implementazione di `equals` di `T2` la condizione `o instanceof T1` con la condizione `getClass() == o.getClass()` che è simmetrica (e risulta vera se e solo se gli oggetti hanno identico tipo concreto).

In [9]:
class T2 {
    private final int a;
    public T2(int a) { this.a = a; }
    public boolean equals(Object o) {
        if (getClass() == o.getClass()) {
            final T2 t = (T2)o;
            return a == t.a;
        }
        return false;
    }
}

class S2 extends T2 {
    private final int b;
    public S2(int a, int b) { super(a); this.b = b; }
    public boolean equals(Object o) {
        if (getClass() == o.getClass()) {
            final S2 s = (S2)o;
            return super.equals(o) && b == s.b;
        }
        return false;
    }
}

S2 s = new S2(1, 2);
T2 t = new T2(1);

In questo modo la simmetria e la transitività sono rispettate perché `equals` funziona solo per oggetti del medesimo tipo, avendo valore `false` per oggetti di tipo diverso (anche se sottotipi l'uno dell'altro).

In [10]:
s.equals(t);

false

In [11]:
t.equals(s);

false

Questo però viola il **principio di sostituzione**! 

Per cogliere meglio questo aspetto consideriamo `S3`, una ulteriore estensione di `T2`che non aggiunga alcun "valore", ma solo un "comportamento".

In [12]:
class S3 extends T2 {
    public S3(int a) { super(a); }
    public boolean equals(Object o) {
        if (o instanceof T2) return super.equals(o);
        return false;
    }
    public void sayHi() { System.out.println("Hi!"); }
}

Supponiamo ora di sviluppare una classe di utilità con un metodo `isSmall` che restituisca `true` per tutti gli oggetti per cui `a` è `1`, oppure `2` e supponiamo (sebbene questo sia estremamente innaturale ed inefficiente) di implementare tale metodo verificando l'appartenenza ad un insieme `SMALLS` che contiene gli unici due oggetti che soddisfano tale requisito.

In [13]:
class U {
    private static final Set<T2> SMALLS = Set.of(new T2(1), new T2(2));
    public static boolean isSmall(T2 t) { return SMALLS.contains(t); }
}

Il principio di sostituzione predica che sostituendo oggetti di sottotipo `S3` ad oggetti di tipo `T2` il comportamento del codice non deve cambiare, ma questo non accade:

In [14]:
T2 v = new T2(1);
S3 u = new S3(1);

In [15]:
U.isSmall(v);

true

In [16]:
U.isSmall(u);

false

Questo accade perché l'implementazione di `Set` si basa sul metodo `equals` per decidere se un oggetto appartiene all'insieme, ma gli elementi di `SMALLS` (che sono di tipo `T2`) non risultano mai uguali ad oggetti di tipo `S3`!